In [1]:
!nvidia-smi

Wed May 18 17:18:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.00    Driver Version: 470.82.00    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:01:00.0 Off |                  Off |
| 30%   49C    P8    42W / 300W |     13MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:2E:00.0 Off |                  Off |
| 80%   

In [2]:
import numpy as np
import torch
import torch.nn as nn
from model import FNN
from util import *
from train import *
from torch.autograd import Variable,grad
import time

In [3]:
def output_transform(X):
    X = T_range*nn.Softplus()(X) + T_ref
    return X

def input_transform(X):
    X = 2.*(X-X_min)/(X_max-X_min) - 1.
    return X

In [4]:
def PDE(x,y,z,t,net):
    
    X = torch.concat([x,y,z,t],axis=-1)
    T = net(X)
    
    T_t = grad(T,t,create_graph=True,grad_outputs=torch.ones_like(T))[0]

    T_x = grad(T,x,create_graph=True,grad_outputs=torch.ones_like(T))[0]
    T_xx = grad(T_x,x,create_graph=True,grad_outputs=torch.ones_like(T_x))[0]
    
    T_y = grad(T,y,create_graph=True,grad_outputs=torch.ones_like(T))[0]
    T_yy = grad(T_y,y,create_graph=True,grad_outputs=torch.ones_like(T_y))[0]
    
    T_z = grad(T,z,create_graph=True,grad_outputs=torch.ones_like(T))[0]
    T_zz = grad(T_z,z,create_graph=True,grad_outputs=torch.ones_like(T_z))[0]
    
    
    Cp = a1*T + b1 + (T>=solidus)*(T<liquidus)*latent/(liquidus-solidus)
    k = a2*T + b2
    
    f = rho*Cp*T_t - k*(T_xx+T_yy+T_zz)

    return f

In [5]:
def BC(x,y,z,t,loc,l=None):
    X = torch.concat([x,y,z,t],axis=-1)
    T = net(X)
    k = a2*T + b2
    if loc == '-x':
        T_x = grad(T,x,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return k*T_x - h*T - Rboltz*emiss*T**4
    if loc == '+x':
        T_x = grad(T,x,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return -k*T_x - h*T - Rboltz*emiss*T**4
    if loc == '-y':
        T_y = grad(T,y,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return k*T_y - h*T - Rboltz*emiss*T**4
    if loc == '+y':
        T_y = grad(T,y,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return -k*T_y - h*T - Rboltz*emiss*T**4
    if loc == '-z':
        T_t = grad(T,t,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return T_t
    if loc == '+z':
        T_z = grad(T,z,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        q = 2*P*eta/torch.pi/r**2*torch.exp(-2*(torch.square(x-x0-v*t)+torch.square(y-y0))/r**2)*(t<t_end)*(z>29.95)
        return -k*T_z - h*T + q

In [6]:
def generate_points(p=[],f=[]):

    t = np.linspace(0,x_max[3],71)
    
    # boundary points
    bound_x_neg,_ = sampling_uniform(1.,x_min,x_max_prev,'-x',t)
    bound_x_pos,_ = sampling_uniform(1.,x_min,x_max,'+x',t)

    bound_y_neg,_ = sampling_uniform(1.,x_min,x_max_prev,'-y',t)

    bound_z_neg,_ = sampling_uniform(1.,x_min,x_max,'-z',t)
    bound_z_pos,_ = sampling_uniform(1.,x_min,x_max,'+z',t)
    bound_z_pos[bound_z_pos[:,0]<x0+bound_z_pos[:,3]*v-r,2] = x_max_prev[2]
    
    bound_x_neg_mov = [] # tracking moving '-x' surface
    bound_y_neg_mov = [] # tracking moving '-y' surface
    bound_z_pos_more = [] # more points for surface flux

    for ti in t:
        xi,_ = sampling_uniform(.25,
                                [max(x0+ti*v-r,x_min[0]),x_min[1],x_max_prev[2]],[x_max[0],x_max[1],x_max[2]],
                                '-x',[ti])
        yi,_ = sampling_uniform(.25,
                                [max(x0+ti*v-r,x_min[0]),x_min[1],x_max_prev[2]],[x_max[0],x_max[1],x_max[2]],
                                '-y',[ti])

        bound_x_neg_mov.append(xi)
        bound_y_neg_mov.append(yi)

        if ti<t_end:
            zi,_ = sampling_uniform(.25,
                        [max(x0+ti*v-r,x_min[0]),max(x_min[1],y0-r),x_min[2]],
                        [min(x0+ti*v+r,x_max[0]),min(x_max[1],y0+r),x_max[2]],
                        '+z',[ti])
            bound_z_pos_more.append(zi)


    bound_x_neg_mov = np.vstack(bound_x_neg_mov)
    bound_y_neg_mov = np.vstack(bound_y_neg_mov)
    bound_z_pos_more = np.vstack(bound_z_pos_more)

    bound_x_neg = np.vstack((bound_x_neg,bound_x_neg_mov))
    bound_y_neg = np.vstack((bound_y_neg,bound_y_neg_mov))
    bound_y_pos = np.copy(bound_y_neg)
    bound_y_pos[:,1] = x_max[1]
    bound_z_pos = np.vstack((bound_z_pos,bound_z_pos_more))
    
    
    ### domain points
    domain_pts1,_ = sampling_uniform([2. , 1. , 2.],
                                     [x_min[0],x_min[1],x_min[2]],
                                     [x_max[0],x_max[1],x_max[2]-5.],'domain',t)

    domain_pts2,_ = sampling_uniform([1.5 ,.5 , 1.],
                                     [x_min[0],x_min[1],x_max[2]-5.],
                                     [x_max[0],x_max[1],x_max[2]-.75],'domain',t)
    
    domain_pts3 = []
    for ti in t:
        di,_ = sampling_uniform([1., 0.25, 0.5],
                                [max(x0+ti*v-r,x_min[0]),x_min[1],x_max[2]-.75,],
                                [x_max[0],x_max[1],x_max[2]],'domain',[ti])
        domain_pts3.append(di)
        
    domain_pts3 = np.vstack(domain_pts3)
    domain_pts = np.vstack((domain_pts1,domain_pts2,domain_pts3))
    
    p.extend([torch.tensor(bound_x_neg,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_x_pos,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_y_neg,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_y_pos,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_z_neg,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_z_pos,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(domain_pts,requires_grad=True,dtype=torch.float).to(device)])
    f.extend([['BC','-x'],['BC','+x'],['BC','-y'],['BC','+y'],['BC','-z'],['BC','+z'],['domain']])
    
    return p,f

In [7]:
def load_data(p=[],f=[]):
    init_data = np.load('../data/init.npy')
    IR_data = np.load('../data/IR_data.npy')
    
    if task == 'time_pred':
        IR_data = IR_data[(IR_data[:,3]>0.5)*(IR_data[:,3]<4)]
    
    if task == 'domain_pred':
        IR_data = IR_data[(IR_data[:,2]>25)]
      
    if task == 'no_data':
        p.extend([torch.tensor(init_data[:,0:4],requires_grad=True,dtype=torch.float).to(device)])
        f.extend([['IC',torch.tensor(init_data[:,4:5],requires_grad=True,dtype=torch.float).to(device)]])
        return p,f

    p.extend([torch.tensor(init_data[:,0:4],requires_grad=True,dtype=torch.float).to(device),
                  torch.tensor(IR_data[:,0:4],requires_grad=True,dtype=torch.float).to(device)])
    
    f.extend([['IC',torch.tensor(init_data[:,4:5],requires_grad=True,dtype=torch.float).to(device)],
            ['data',torch.tensor(IR_data[:,4:5],requires_grad=True,dtype=torch.float).to(device)]])
    
    return p,f

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
x_max = np.array([ 22.5, 1.25, 30., 7.])
x_min = np.array([-22.5,-1.25,  0.,  0.])
x_max_prev = np.array([ 22.5, 1.25, 29.25, 7.])
X_max = torch.tensor(x_max,dtype=torch.float).to(device)
X_min = torch.tensor(x_min,dtype=torch.float).to(device)

a1 = 2.0465e-4
b1 = 3.8091e-1
a2 = 1.6702e-5
b2 = 5.5228e-3
latent = 250
liquidus = 1609
solidus = 1533

T_ref = 298
T_range = 3000

x0 = 22.5 # laser start x
y0 = 0
r = 1.12 # beam radius
v = -7 # speed
t_end = 45/7 # laser end time

task = 'no_data'

h = 2e-5
eta = 0.48
P = 500
Rboltz = 5.6704e-14
emiss = 0.3
rho = 8.19e-3

In [ ]:
iterations = 50000

net = FNN([4,64,64,64,1],nn.Tanh(),in_tf=input_transform,out_tf=output_transform)
net.to(device)

point_sets,flags = generate_points([],[])
point_sets,flags = load_data(point_sets,flags)

l_history = train(net,PDE,BC,point_sets,flags,iterations,lr=5e-4,info_num=100)

It: 0, Loss: 6.697e+01, L_bc: 8.120e+01, L_ic: 1.818e+06, L_data: 0.000e+00, L_PDE: 4.880e+00, Time: 0.12
It: 100, Loss: 1.371e+01, L_bc: 5.185e+01, L_ic: 2.731e+04, L_data: 0.000e+00, L_PDE: 2.491e-01, Time: 10.07
It: 200, Loss: 1.340e+01, L_bc: 4.932e+01, L_ic: 4.062e+04, L_data: 0.000e+00, L_PDE: 2.365e-01, Time: 10.08
It: 300, Loss: 1.320e+01, L_bc: 4.816e+01, L_ic: 4.353e+04, L_data: 0.000e+00, L_PDE: 2.785e-01, Time: 10.09
It: 400, Loss: 1.300e+01, L_bc: 4.740e+01, L_ic: 4.237e+04, L_data: 0.000e+00, L_PDE: 3.452e-01, Time: 10.10
It: 500, Loss: 1.267e+01, L_bc: 4.665e+01, L_ic: 3.611e+04, L_data: 0.000e+00, L_PDE: 4.322e-01, Time: 10.10
It: 600, Loss: 1.190e+01, L_bc: 4.497e+01, L_ic: 2.099e+04, L_data: 0.000e+00, L_PDE: 5.325e-01, Time: 10.11
It: 700, Loss: 1.106e+01, L_bc: 4.273e+01, L_ic: 8.826e+03, L_data: 0.000e+00, L_PDE: 6.165e-01, Time: 10.11
It: 800, Loss: 1.066e+01, L_bc: 4.118e+01, L_ic: 7.808e+03, L_data: 0.000e+00, L_PDE: 6.767e-01, Time: 10.12
It: 900, Loss: 1.027e+

It: 7500, Loss: 1.126e+00, L_bc: 1.685e+00, L_ic: 4.568e+03, L_data: 0.000e+00, L_PDE: 2.362e+00, Time: 10.13
It: 7600, Loss: 1.131e+00, L_bc: 1.651e+00, L_ic: 4.502e+03, L_data: 0.000e+00, L_PDE: 2.421e+00, Time: 10.13
It: 7700, Loss: 1.170e+00, L_bc: 1.731e+00, L_ic: 4.426e+03, L_data: 0.000e+00, L_PDE: 2.506e+00, Time: 10.13
It: 7800, Loss: 1.136e+00, L_bc: 1.753e+00, L_ic: 4.554e+03, L_data: 0.000e+00, L_PDE: 2.334e+00, Time: 10.14
It: 7900, Loss: 1.159e+00, L_bc: 1.748e+00, L_ic: 4.572e+03, L_data: 0.000e+00, L_PDE: 2.430e+00, Time: 10.13
It: 8000, Loss: 1.109e+00, L_bc: 1.670e+00, L_ic: 4.405e+03, L_data: 0.000e+00, L_PDE: 2.325e+00, Time: 10.13
It: 8100, Loss: 1.121e+00, L_bc: 1.666e+00, L_ic: 4.338e+03, L_data: 0.000e+00, L_PDE: 2.385e+00, Time: 10.13
It: 8200, Loss: 1.116e+00, L_bc: 1.691e+00, L_ic: 4.322e+03, L_data: 0.000e+00, L_PDE: 2.342e+00, Time: 10.13
It: 8300, Loss: 1.190e+00, L_bc: 1.641e+00, L_ic: 4.133e+03, L_data: 0.000e+00, L_PDE: 2.706e+00, Time: 10.14
It: 8400, 

In [ ]:
torch.save(net.state_dict(),'../model/{}.pt'.format(task))
np.save('../model/{}.npy'.format(task),l_history)